In [1]:
#required installations.
!pip install sentence-transformers faiss-cpu python-docx
!pip install -q gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

In [2]:
#required imports.
from docx import Document
from sentence_transformers import SentenceTransformer
import faiss
import pickle

model = SentenceTransformer('all-mpnet-base-v2') #SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
loaded_entries=[]
loaded_index=None
file_path = 'Problem.docx'


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [3]:
def LoadDocData(file_path):
  doc = Document(file_path)
  document_content = []
  for paragraph in doc.paragraphs:
      document_content.append(paragraph.text)

  # Loading the document entries by grouping Problem, Solution, and details
  entries = []
  current_entry = []
  for paragraph in document_content:
      if paragraph.strip():  # Checking if the paragraph is not empty or just whitespace
          if paragraph.strip().startswith("Problem:"):
              if current_entry: # If there's a previous entry, add it to entries
                  entries.append("\n".join(current_entry))
              current_entry = [paragraph.strip()] # Start a new entry
          else:
              current_entry.append(paragraph.strip()) # Add to the current entry

  # Add the last entry after the loop finishes
  if current_entry:
      entries.append("\n".join(current_entry))


  #creating embeddings for document entries
  # Load the model


  # Generating embeddings for each entry
  entry_embeddings = model.encode(entries)

  #Initializing a FAISS index with the correct embedding dimension is crucial because FAISS works by comparing vectors in that specific dimensional space.
  #If the dimension of the index doesn't match the dimension of our embeddings, FAISS won't be able to correctly process the vectors,
  #leading to errors or incorrect search results. the data won't align properly for the search to work.

  # Get the dimension of the embeddings
  embedding_dimension = entry_embeddings.shape[1]

  # Initialize a IndexFlatL2 FAISS index
  index = faiss.IndexFlatL2(embedding_dimension)

  # We have to add the generated embeddings to the FAISS index to create a searchable databse for our document entries
  #once the embeddings are added we can efficiently perform simlarity searches. (this allows for fast and scalable nearest
  # neighbors search which is required for finding duplicate entries or related information)

  # Add the embeddings to the index
  index.add(entry_embeddings)

  # Saving the FAISS index and the corresponding text entries to disk for later use.
  # Define file paths
  index_file = 'faiss_index.bin'
  entries_file = 'entries.pkl' # Using pickle to save list

  # Save the FAISS index
  faiss.write_index(index, index_file)
  print(f"FAISS index saved to {index_file}")

  # Save the entries list
  with open(entries_file, 'wb') as f:
      pickle.dump(entries, f)




In [4]:
def LoadEntries():
  index_file = 'faiss_index.bin'
  entries_file = 'entries.pkl'

  # Loading the FAISS index
  loaded_index = faiss.read_index(index_file)

  # Load the entries list
  with open(entries_file, 'rb') as f:
      loaded_entries = pickle.load(f)

  return loaded_index, loaded_entries

In [5]:
LoadDocData(file_path)
loaded_index, loaded_entries = LoadEntries()


FAISS index saved to faiss_index.bin


In [9]:
# required utility functions.

# function to add entry to the document.
def append_to_docx(problem_text, solution_text):
    """Appends a new problem and solution to the 'problem.docx' file.

    Args:
        problem_text: The text of the problem.
        solution_text: The text of the solution.
    """
    try:
        doc = Document(file_path)
    except: # If the file doesn't exist, create a new one
        doc = Document()
        doc.add_paragraph("## Problem and Solution Entries")


    # Add the problem and solution
    doc.add_paragraph(f"Problem: {problem_text}")
    doc.add_paragraph(f"Solution: {solution_text}")

    # Save the document
    doc.save(file_path)
    #LoadDocData(file_path)
    #Lodaded_index, loaded_entries = LoadEntries()

# function to update an existing entry in the document.
def update_docx_entry(index, new_problem_text, new_solution_text):
    """Updates an existing problem and solution entry in the 'problem.docx' file.

    Args:
        index: The index of the entry to update (based on the loaded_entries list).
        new_problem_text: The new text for the problem.
        new_solution_text: The new text for the solution.
    """
    try:
        doc = Document(file_path)
        paragraphs = doc.paragraphs

        # Find the paragraphs corresponding to the entry at the given index.
        # Each entry consists of a "Problem:" paragraph and a "Solution:" paragraph.
        # The index in loaded_entries corresponds to the problem paragraph.
        # So, the problem paragraph is at index * 2, and the solution is at index * 2 + 1,
        # considering the initial "## Problem and Solution Entries" paragraph.
        problem_paragraph_index = index * 2 + 1
        solution_paragraph_index = index * 2 + 2

        if solution_paragraph_index < len(paragraphs):
            # Update the text of the problem and solution paragraphs
            paragraphs[problem_paragraph_index].text = f"Problem: {new_problem_text}"
            paragraphs[solution_paragraph_index].text = f"Solution: {new_solution_text}"
            doc.save(file_path)
            print(f"Entry at index {index} updated in Problem.docx")
            #LoadDocData(file_path)
            #Lodaded_index, loaded_entries = LoadEntries()
        else:
            print(f"Error: Index {index} out of range for updating.")

    except Exception as e:
        print(f"Error updating document: {e}")


# function to convert text to embeddings
def generate_embedding(text):
  """Generates an embedding for the input text using the loaded model.

  Args:
    text: The input text string.

  Returns:
    A numpy array representing the embedding of the text.
  """
  # Encode the text. model.encode expects a list.
  embedding = model.encode([text])
  # Return the first (and only) row of the embedding array
  return embedding[0]

# Semantic search function
def perform_semantic_search(embedding, k):
  """Performs a semantic search against the loaded FAISS index.

  Args:
    embedding: The embedding of the query text (numpy array).
    k: The number of nearest neighbors to retrieve.

  Returns:
    A tuple containing:
      - distances: A numpy array of distances to the nearest neighbors.
      - indices: A numpy array of indices of the nearest neighbors in the loaded_entries list.
  """
  # Reshape the embedding to match the expected input shape of index.search (1, dimension)
  embedding = embedding.reshape(1, -1)
  distances, indices = loaded_index.search(embedding, k)
  return distances, indices

# Function to call the LLM for comparison
import google.generativeai as genai
from google.colab import userdata

def compare_with_llm(new_entry, similar_entries):
    """
    Compares the new entry with similar entries using an LLM.

    Args:
        new_entry: The text of the new problem and solution.
        similar_entries: A list of text strings of similar entries from the document.

    Returns:
        A string containing the LLM's assessment.
    """
    # Configure the Gemini API
    GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)

    # Initialize the Generative Model
    # Using gemini-1.5-flash-latest as gemini-flash-2 is not a valid model name
    # You can change this to the specific model you want to use if it's available
    gemini_model = genai.GenerativeModel('gemini-2.0-flash') #('gemini-1.5-flash-latest')

    # Create the prompt for the LLM
    #prompt = f"""Compare the following new entry with the existing similar entries.
    #Determine which entry provides the most accurate and informative solution to the problem.
    #Provide a brief explanation for your choice.

    prompt = f"""Compare the following new entry with the existing similar entries.
    Determine which has more information in the solution.
    if new entry has more information then return response as True else False.


    New Entry:
    {new_entry}

    existing similar Entries:
    {'- '.join(similar_entries)}

    Which entry is better and why?
    """

    # Call the LLM
    try:
        response = gemini_model.generate_content(prompt)
        return response.text.strip() # Strip whitespace from the response
    except Exception as e:
        return f"Error calling LLM: {e}"


def process_problem_solution(problem, solution):
    """
    Processes the input problem and solution, performs semantic search,
    and updates the document based on similarity and LLM comparison.

    Args:
        problem: The input problem string from the UI.
        solution: The input solution string from the UI.

    Returns:
        A tuple containing:
            - similarity_score_str: The calculated similarity score as a formatted string.
            - action_taken: A string describing the action taken (added, ignored, or compared with LLM).
            - comparison_result: A string containing the LLM comparison result (if performed).
    """
    # 2. Combine the problem and solution into a single string
    new_entry = f"Problem: {problem}\nSolution: {solution}"

    # 3. Use the generate_embedding function to get the embedding
    new_entry_embedding = generate_embedding(new_entry)

    # 4. Use the perform_semantic_search function to find the most similar existing entry
    # We need the top 1 result (k=1)
    k = 3
    distances, indices = perform_semantic_search(new_entry_embedding, k)

    # Extract the text of the similar entries
    similar_entries = [loaded_entries[i] for i in indices[0]]
    # Get the index of the most similar entry
    most_similar_index = indices[0][0]


    # 5. Calculate the similarity score from the returned distance for the top result.
    # Assuming L2 normalized embeddings, cosine similarity = 1 - ||a - b||^2 / 2
    # distances[0][0] is the squared Euclidean distance for the top result
    top_distance_squared = distances[0][0]
    similarity_score = 1 - (top_distance_squared / 2)

    # 6. Define the similarity_threshold
    similarity_threshold = 0.70

    comparison_result = "" # Initialize comparison result

    # 7. & 8. Compare the calculated similarity score with the similarity_threshold
    if similarity_score < similarity_threshold:
        # If similarity is below the threshold, add the new entry
        append_to_docx(problem, solution)
        action_taken = "Entry added to the knowledge base."
    else:
        # If similarity is at or above the threshold, a similar entry exists
        action_taken = "Similar entry found. Comparing with LLM."
        # Perform LLM comparison
        comparison_result = compare_with_llm(new_entry, similar_entries)

        # If comparison_result contains 'True', replace the existing similar content with new_entry
        if "True" in comparison_result:
             update_docx_entry(most_similar_index, problem, solution)
             action_taken += " Existing entry updated with new entry based on LLM comparison."
        else:
             action_taken += " Existing entry is better or equally good based on LLM comparison. New entry not added."


    # Format the similarity score for display
    similarity_score_str = f"{similarity_score:.4f}"


    # 9. Return the calculated similarity_score, the action_taken message, and the LLM comparison result
    return similarity_score_str, action_taken, comparison_result

In [ ]:
import gradio as gr
from IPython.display import display

# Define the Gradio interface
interface = gr.Interface(
    fn=process_problem_solution,  # Your processing function
    inputs=[
        gr.Textbox(lines=5, label="Problem"),
        gr.Textbox(lines=5, label="Solution")
    ],
    outputs=[
        gr.Textbox(label="Similarity Score"),
        gr.Textbox(label="Action taken"),
        gr.Textbox(label="LLM Comparison Result")
    ],
    title="Problem and Solution Processor",
    description="Enter a problem and its solution to get processed results."
)

# Launch the interface
interface.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://ce523082cc8bb14736.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Entry at index 2 updated in Problem.docx


In [10]:
prb ='Email Not Sending from Outlook'
sln='Check SMTP server settings. Make sure the internet connection is stable. Clear the Outbox and restart Outlook. Re-enter password if account was recently updated. Repair Outlook profile via Control Panel > Mail.'
r1,r2,r3 = process_problem_solution(prb,sln)

print(r1)
print(r2)
print(r3)

Entry at index 2 updated in Problem.docx
0.8576
Similar entry found. Comparing with LLM. Existing entry updated with new entry based on LLM comparison.
The new entry has more information in the solution compared to the existing similar entry.

**Response: True**

**Explanation:**

The new entry's solution offers a more comprehensive and detailed set of troubleshooting steps specifically for the "Email Not Sending from Outlook" problem.  It includes actions like:

*   Clearing the Outbox
*   Restarting Outlook
*   Re-entering the password
*   Repairing the Outlook profile.

The original similar entry only had one point: "Check SMTP server settings."

Therefore, the new entry provides a more helpful and informative solution for the user.
